In [1]:
# Import libraries
import xml.etree.ElementTree as ET #parse xml file
import pandas as pd 
import nltk
from nltk import word_tokenize, pos_tag

/Users/subrata.a.ghosh/Library/Python/3.6/lib/python/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/subrata.a.ghosh/.matplotlib/matplotlibrc", line #2
  (fname, cnt))
/Users/subrata.a.ghosh/Library/Python/3.6/lib/python/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/subrata.a.ghosh/.matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
def parse_xml(root):
    all_aspectTerms = [] #only aspect terms 
    all_text_with_boi = [] #review with aspect terms

    for sentences in root.findall('sentence'):
        text = sentences.find('text').text
        aspectTerms = sentences.find('aspectTerms')
        asp = []
        try: #Because, review text might not any aspect term
            for neighbor in aspectTerms.iter('aspectTerm'):
                asp1 = neighbor.attrib.get('term',"")
                #all_aspectTerms.append(asp1)
                asp.append(asp1)
            as_word_boi = BIO(text, asp)
            all_aspectTerms.append(asp)
            all_text_with_boi.append(as_word_boi)
        except:
            as_word_boi = BIO(text, asp)
            all_text_with_boi.append(as_word_boi)
            all_aspectTerms.append(asp)


    return (all_text_with_boi,all_aspectTerms)  

In [3]:
def BIO(test_text, aspect_terms):
    all_word = {}
    #split_test = word_tokenize(test_text)
    split_test = test_text.split()
    pos_tag_ = pos_tag(split_test)
    for word in split_test:
        all_word[word] = "O"
    if len(aspect_terms)>0:
        for aspect in aspect_terms:
            fl=0
            for asp in aspect.split():
                for word in split_test:
                    if asp.lower() in word.lower():
                        #print (asp,word)
                        asp = word
                        break
                if fl==0:            
                    all_word[asp] = "B" 
                    fl = 1
                else:
                    all_word[asp] = "I"
    li = []
    for word, cls in all_word.items():
        for word1,tag in pos_tag_:
            if word == word1:
                li.append((word,cls,tag))
                break
    return li

In [4]:
train_tree = ET.parse('Laptops_Train_v2.xml')
train_root = train_tree.getroot()

training,all_aspectTerms = parse_xml(train_root)

print (training[0]) # format = (review text, [aspect trams,....])Restaurants_Train.xml

[('I', 'O', 'PRP'), ('charge', 'O', 'VBP'), ('it', 'O', 'PRP'), ('at', 'O', 'IN'), ('night', 'O', 'NN'), ('and', 'O', 'CC'), ('skip', 'O', 'NN'), ('taking', 'O', 'VBG'), ('the', 'O', 'DT'), ('cord', 'B', 'NN'), ('with', 'O', 'IN'), ('me', 'O', 'PRP'), ('because', 'O', 'IN'), ('of', 'O', 'IN'), ('good', 'O', 'JJ'), ('battery', 'B', 'NN'), ('life.', 'I', 'NN')]


In [5]:
test_tree = ET.parse('Laptops_Test_Gold.xml')
test_root = test_tree.getroot()

test,all_aspectTerms = parse_xml(test_root)

print (test[0]) # format = (review text, [aspect trams,....])
print (len(all_aspectTerms))

[('Boot', 'B', 'NNP'), ('time', 'I', 'NN'), ('is', 'O', 'VBZ'), ('super', 'O', 'JJ'), ('fast,', 'O', 'NN'), ('around', 'O', 'RB'), ('anywhere', 'O', 'RB'), ('from', 'O', 'IN'), ('35', 'O', 'CD'), ('seconds', 'O', 'NNS'), ('to', 'O', 'TO'), ('1', 'O', 'CD'), ('minute.', 'O', 'NNS')]
800


In [7]:
def features(tagged, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    sentence = [tag for tag,a,b in tagged]
    return {
        'word': sentence[index],
        'tag': tagged[index][2],
        'prev_prev_tag': '' if index <= 2 else tagged[index - 2][2],
        'prev_tag': '' if index == 0 else tagged[index - 1][2],
        'next_tag': '' if index == len(sentence) - 1 else tagged[index + 1][2],
        'next_next_tag': '' if index >= len(sentence) - 2 else tagged[index + 2][2],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

In [18]:
#from nltk.tag.util import untag
def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        X.append([features(tagged, index) for index in range(len(tagged))])
        y.append([a for _,a,tag in tagged])
 
    return X, y
 
X_train, y_train = transform_to_dataset(training)
X_test, y_test = transform_to_dataset(test)
 
print(len(X_train))     
print(len(X_test))         
print(X_train[0])
print(y_train[0])

3045
800
[{'word': 'I', 'tag': 'PRP', 'prev_prev_tag': '', 'prev_tag': '', 'next_tag': 'VBP', 'next_next_tag': 'PRP', 'is_first': True, 'is_last': False, 'is_capitalized': True, 'is_all_caps': True, 'is_all_lower': False, 'prefix-1': 'I', 'prefix-2': 'I', 'prefix-3': 'I', 'suffix-1': 'I', 'suffix-2': 'I', 'suffix-3': 'I', 'prev_word': '', 'next_word': 'charge', 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': 'charge', 'tag': 'VBP', 'prev_prev_tag': '', 'prev_tag': 'PRP', 'next_tag': 'PRP', 'next_next_tag': 'IN', 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': True, 'prefix-1': 'c', 'prefix-2': 'ch', 'prefix-3': 'cha', 'suffix-1': 'e', 'suffix-2': 'ge', 'suffix-3': 'rge', 'prev_word': 'I', 'next_word': 'it', 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': 'it', 'tag': 'PRP', 'prev_prev_tag': '', 'prev_tag': 'VBP', 'next_tag': 'IN', 'next_next_tag': 'NN', 'is_first': False, 'is

In [9]:
from sklearn_crfsuite import CRF
 
model = CRF(
    algorithm='l2sgd',
    #c1=0.1,
    c2=0.01,
    #period=20,
    calibration_eta=0.01,
    max_iterations=3000,
    all_possible_transitions=True,
    #all_possible_states=True 
)
model.fit(X_train, y_train)

CRF(algorithm='l2sgd', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=0.01,
  calibration_candidates=None, calibration_eta=0.01,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=3000,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [10]:
model

CRF(algorithm='l2sgd', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=None, c2=0.01,
  calibration_candidates=None, calibration_eta=0.01,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=3000,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [11]:
from sklearn_crfsuite import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,accuracy_score

y_pred = model.predict(X_test)
print(metrics.flat_accuracy_score(y_test, y_pred))
print(metrics.flat_classification_report(y_test, y_pred))

0.9452082905807511
              precision    recall  f1-score   support

           B       0.78      0.61      0.68       639
           I       0.85      0.49      0.62       393
           O       0.96      0.99      0.97      8714

   micro avg       0.95      0.95      0.95      9746
   macro avg       0.86      0.70      0.76      9746
weighted avg       0.94      0.95      0.94      9746



In [349]:
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

In [154]:

%%time
# define fixed parameters and parameters to search
crf = CRF(
    algorithm='l2sgd',
    max_iterations=3000,
    all_possible_transitions=True,
    all_possible_states=True
)
params_space = {
    'calibration_eta': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted')

# search
model = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
model.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 14.9min finished


CPU times: user 13min 40s, sys: 8.41 s, total: 13min 48s
Wall time: 14min 59s


In [155]:
# crf = rs.best_estimator_
print('best params:', model.best_params_)
print('best CV score:', model.best_score_)
print('model size: {:0.2f}M'.format(model.best_estimator_.size_ / 1000000))

best params: {'c2': 0.11408738555005796, 'calibration_eta': 1.0620521205678686}
best CV score: 0.9506949414571738
model size: 3.17M


In [156]:
model = model.best_estimator_
y_pred = model.predict(X_test)

In [16]:
all_pred_asp = []
for i in range(len(y_pred)):
    asp_list = []
    asp = ""
    fl = 0
    if(359):
        for j in range(len(y_pred[i])):
            if y_pred[i][j] == "B" and fl ==0:
                asp += test[i][j][0]
                fl=1
                #print(asp)
            elif(y_pred[i][j] == "B" and fl ==1):
                asp_list.append(asp)
                asp = ""
                asp += test[i][j][0]
                fl=1
            elif(y_pred[i][j] == "I" and fl==1):
                asp += " "+ test[i][j][0]
                #print(asp)
            elif(y_pred[i][j] == "I" and fl==0):
                asp += test[i][j][0]
                fl=1
            else:
                if fl==1:
                    asp_list.append(asp)
                fl=0
                asp = ""
        if fl==1:
            asp_list.append(asp)
        all_pred_asp.append(asp_list)
    

In [17]:
c = 0
p = 0
r = 0
f = open("crf_asp.txt",'w')
for i in range(len(all_pred_asp)):
    pre = [x.lower().replace(".", "").replace(",", "") for x in all_pred_asp[i]]
    act = [x.lower() for x in all_aspectTerms[i]]
    #for asp in pre:
        #if asp in act:
            #c+=1
    c += len([a for a in pre if a in act])
    #print (pre)
    #print (act)
    p += len(all_pred_asp[i])
    r += len(all_aspectTerms[i])
    
    ss = ""
    for token in pre:
        ss+=token+'/'
    f.write(ss)
    f.write('\n')
    
    #break
f.close()
print (c)
print (float(c/p))
print (float(c/r))
print ((2*float(c/p)*float(c/r))/(float(c/p)+float(c/r)))

354
0.708
0.5412844036697247
0.6135181975736569
